Difficultées avec MtSup : 

Ceux en forme de sous dict a été rapidement réglé, mais il existe sous forme de liste contenant deux codes et deux vals. 

  'MtSup': "[{'@Code': 'ProdChaRat', '@V': '1670.26, {'@Code': 'BudgetHorsRAR', '@V': '104379.00]", C'EST MEME PAS DES DICTIONNAIRES

Plusieurs options : Faire deux lignes (très risqué) créer deux nouvelles colonnes (il peut en exister qui nécessite 3 col etc etc)

In [12]:
import gzip 
import xmltodict
import pandas as pd
from lxml import etree 

In [2]:
objet_xml = "./todo/20200804-1814937.xml.gz"
arbre = etree.parse(objet_xml)
racine = arbre.getroot()

In [3]:
coeur = racine.getchildren()
entete = coeur[4]
date = entete.getchildren()
liste_meta = []
for i in date : 
 a = str(i).split("}")[-1].split(' ')[0]
 liste_meta.append(a)
liste_meta
valeurs_meta = []
for i in date : 
 b = i.attrib
 valeurs_meta.append(str(b).split(' ')[-1].split("'}")[0])
valeurs_meta


["'2020-04-22",
 'BONNEVILLE',
 "'074009",
 "'BONNEVILLE",
 "'21740042300018",
 '',
 '']

In [ ]:
dict_metadonnees = {}
for i in date : 
 a = str(i).split("}")[-1].split(' ')[0]
 b = str(i.attrib).split(" '")[-1].split("'}")[0]
 dict_metadonnees[a] = b
nomenclature = coeur[5].getchildren()[0].getchildren()[5]
dict_metadonnees['Nomenclature'] = str(nomenclature.attrib).split(" '")[-1].split("'}")[0]
dict_metadonnees = {a: b for a, b in dict_metadonnees.items() if b != '}'}
dict_metadonnees

In [4]:

def parsing_fichier(chemin) : 
 with gzip.open(chemin, 'rb') as fichier_ouvert : 
  fichier_xml_gzip = fichier_ouvert.read()
  fichier_xml = fichier_xml_gzip.decode('latin-1')
  fichier_dict = xmltodict.parse(fichier_xml)
 return fichier_dict

def _extract_ligne_budget(data_dict: dict) -> pd.DataFrame : 
 "Sépare les sous clefs lignes budget, sans nettoyage"
 ligne_budget = data_dict['DocumentBudgetaire']['Budget']['LigneBudget']
 return ligne_budget

In [5]:
fichier_parsé = parsing_fichier(objet_xml)
fichier_parsév2 = parsing_fichier(objet_xml).copy()

In [6]:
fichier_parsé

{'DocumentBudgetaire': {'@xmlns': 'http://www.minefi.gouv.fr/cp/demat/docbudgetaire',
  'VersionSchema': {'@V': '88'},
  'VersionOutil': [{'@outil': 'DSC_COMMON', '@version': '27'},
   {'@outil': 'REMAT', '@version': '5_2019'}],
  'Scellement': {'@md5': 'F329FA54295CF8885965E21F6058B89E',
   '@sha1': '123D9254501B8D30275049F6B6645369441F2A69',
   '@date': '2020-08-04T09:54:36'},
  'EnTeteDocBudgetaire': {'DteStr': {'@V': '2020-04-22'},
   'LibellePoste': {'@V': 'TRESORERIE DE BONNEVILLE'},
   'IdPost': {'@V': '074009'},
   'LibelleColl': {'@V': 'BONNEVILLE'},
   'IdColl': {'@V': '21740042300018'},
   'NatCEPL': {'@V': ' '},
   'Departement': {'@V': ' '}},
  'Budget': {'EnTeteBudget': {'LibelleEtab': {'@V': 'VILLE DE BONNEVILLE'},
    'IdEtab': {'@V': '21740042300018'},
    'CodColl': {'@V': '100'},
    'CodInseeColl': {'@V': ' '},
    'CodBud': {'@V': '00'},
    'Nomenclature': {'@V': 'M14-M14_COM_SUP3500'}},
   'BlocBudget': {'Affect': {'@V': ' '},
    'NatDec': {'@V': '09'},
    'Exe

In [ ]:
test_1 = lignes_budget[0]
test_1
mtsup_data = test_1['MtSup']
test_1['MtSup_Code'] = {'@V' : mtsup_data['@Code']}
test_1['MtSup_Val'] = {'@V' : mtsup_data['@V']}
#del test_1['MtSup']
test_1.items()

Ne prend pas en compte les    'MtSup': [{'@Code': 'ProdChaRat', '@V': '1670.26'},
      {'@Code': 'BudgetHorsRAR', '@V': '104379.00'}]}, è.é
      

In [ ]:
def v2extract_ligne_budget(data_dict: dict) -> pd.DataFrame : 
 "Sépare les sous clefs lignes budget, sans nettoyage"
 ligne_budget = data_dict['DocumentBudgetaire']['Budget']['LigneBudget']
 for i in ligne_budget :
  dict_MtSup = i['MtSup']
  i['MtSup_Libelle'] = {'@V' : dict_MtSup['@Code']}
  i['MtSup_Val'] = {'@V' : dict_MtSup['@V']}
  del i['MtSup']
 ligne_budget_df = pd.DataFrame(ligne_budget)
 return ligne_budget_df

ligne_v2 = v2extract_ligne_budget(fichier_parsév2)

In [10]:
def v3extract_ligne_budget(data_dict) :
 ligne_budget = data_dict['DocumentBudgetaire']['Budget']['LigneBudget']
 for i in ligne_budget :
  type_MtSup = i.get('MtSup')
  if isinstance(type_MtSup, dict) : 
   dict_MtSup = i['MtSup']
   i['MtSup_1_Lib'] = {'@V' : dict_MtSup['@Code']}
   i['MtSup_1_Val'] = {'@V' : dict_MtSup['@V']}
  elif isinstance(type_MtSup, list) :
   dict_MtSup = i['MtSup']
   mtsup_propre = {}
   for z, entry in enumerate(dict_MtSup, start=1):
    code = f'MtSup_{z}_Lib'
    valeur = f'MtSup_{z}_Val'
    mtsup_propre[code] = entry['@Code']
    mtsup_propre[valeur] = entry['@V'] 
   i.update(mtsup_propre)
 return ligne_budget



ligne3 = v3extract_ligne_budget(fichier_parsé)
dfligne3 = pd.DataFrame(ligne3)
dfligne3 = dfligne3.drop(columns=['MtSup'])

In [ ]:
def nettoyage_lambda(df : pd.DataFrame) -> pd.DataFrame : 
 "Nettoie les données pour se débarasser des @V"
 nettoyage = lambda x : str(x).replace("{'@V': '", "").replace("'}", "")
 for col in df.columns : 
  df[col] = df[col].apply(nettoyage)
 return df 

dfligne3 = nettoyage_lambda(dfligne3)
dfligne3

GESTION CARAC SUP

In [13]:
objet_xml_carac = "./todo/20200807-1825601.xml.gz"
carac_parse = parsing_fichier(objet_xml_carac)
carac_parse

{'DocumentBudgetaire': {'@xmlns': 'http://www.minefi.gouv.fr/cp/demat/docbudgetaire',
  'VersionSchema': {'@V': '88'},
  'BlocEditeur': {'@xmlns:bl': 'http://www.berger-levrault.fr/demat/outils',
   'CodeEditeur': {'@V': 'Berger-Levrault'},
   'bl:ProduitGF': {'@V': ' - '},
   'bl:EchangeXML': {'@V': '2017.01'}},
  'VersionOutil': [{'@outil': 'DSC_COMMON', '@version': '27'},
   {'@outil': 'REMAT', '@version': '5_2019'}],
  'Scellement': {'@md5': '8746F23B3E49545C5BFCD35A4359759A',
   '@sha1': '6FA2ABA9C8EE7E28EACCDD2F7C01B4C644D429D9',
   '@date': '2020-07-08T15:52:38'},
  'EnTeteDocBudgetaire': {'DteStr': {'@V': '2020-06-11'},
   'LibellePoste': {'@V': 'TRESORERIE DINAN'},
   'IdPost': {'@V': '022025'},
   'LibelleColl': {'@V': 'DINAN AGGLOMERATION'},
   'IdColl': {'@V': '20006898900140'},
   'NatCEPL': {'@V': 'EPCI'},
   'Departement': {'@V': '22'}},
  'Budget': {'EnTeteBudget': {'LibelleEtab': {'@V': 'BUDGET PRINCIPAL'},
    'IdEtab': {'@V': '20006898900140'},
    'CodColl': {'@V': 

In [19]:
nomenclature = carac_parse['DocumentBudgetaire']['Budget']['EnTeteBudget']['Nomenclature']
nomenclature['@V']

'M14-M14_COM_SUP3500'

In [18]:
entetedoc = carac_parse['DocumentBudgetaire']['EnTeteDocBudgetaire']
entetedoc

{'DteStr': {'@V': '2020-06-11'},
 'LibellePoste': {'@V': 'TRESORERIE DINAN'},
 'IdPost': {'@V': '022025'},
 'LibelleColl': {'@V': 'DINAN AGGLOMERATION'},
 'IdColl': {'@V': '20006898900140'},
 'NatCEPL': {'@V': 'EPCI'},
 'Departement': {'@V': '22'}}